# What are the demographic characteristics of neighborhoods where entitlements are?

In [1]:
import json
import warnings
warnings.filterwarnings("ignore")

import geopandas
import intake
import ipyleaflet
import IPython.display
import ipywidgets
import matplotlib.pyplot as plt
import numpy
import pandas

import laplan
import utils

cat = intake.open_catalog("../catalogs/*.yml")

In [2]:
prefix_list = laplan.pcts.VALID_PCTS_PREFIX
suffix_list = laplan.pcts.VALID_PCTS_SUFFIX

remove_prefix = ["ENV"]
remove_suffix = [
    "EIR",
    "IPRO",
    "CA",
    "CATEX",
    "CPIO",
    "CPU",
    "CRA"
    "FH",
    "G",
    "HD",
    "HPOZ",
    "ICO",
    "K",
    "LCP",
    "NSO",
    "RFA",
    "S",
    "SN",
    "SP",
    "ZAI",
]

prefix_list = [x for x in prefix_list if x not in remove_prefix]
suffix_list = [x for x in suffix_list if x not in remove_suffix]

In [3]:
# Census tracts
tracts = cat.census_tracts.read()[["GEOID", "geometry"]].set_index("GEOID")

In [4]:
# Merge the census data with the entitlements counts:
joined, big_cases = utils.entitlements_per_tract(
    prefix_list=prefix_list,
    suffix_list=suffix_list,
    verbose=True,
    big_case_threshold=20,
    return_big_cases=True,
)

Loading PCTS
Parsing PCTS case numbers
6 cases failed to parse.
Getting dummy indicators for case types
Prefixes with no associated cases:  {'HPO'}
Suffixes with no associated cases:  {'ND', 'REC5', 'SCEA', 'REC4', 'EAF', 'SCPE', 'ADD1', 'REC2', 'REC3', 'SUP1', 'SE'}
Loading census analysis table
Loading parcel-tracts crosswalk
Removing cases touching more than 20 parcels
Aggregating entitlements to tract
Joining entitlements to census data


In [5]:
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.CartoDB.Positron)
m.center = [34.07996230865876, -118.31123326410754]
m.zoom = 10

label = ipywidgets.HTML(value=f"<i>Hover to select</i>")
m.add_control(ipyleaflet.WidgetControl(widget=label, position="topright"))

# Plot entitlement stats against median household income,
# population density, and geography:
cols = ["medhhincome", "density", "pct_whitenonhisp", "pct_pop_renter"]

def plot_entitlement(df, tracts, suffix, min_year=2010, max_year=2019):
    to_plot = df[(df[suffix] != 0) & (df.year >= min_year) & (df.year <= max_year)]
    to_plot = to_plot.groupby(to_plot.index).agg({
        suffix: "sum",
        **{c: "first" for c in cols}
    })
    # Merge in geometry
    final_df = tracts.merge(
        to_plot,
        left_index=True,
        right_index=True,
        how="left",
    ).fillna(
        {suffix: 0, **{c: 0 for c in cols}}
    ).to_crs(epsg=4326)
    choro_data = final_df[suffix].to_dict()
    choro_data = {str(x): y for x,y in choro_data.items()}
    geo_data = json.loads(final_df.to_json())
    choro_layer = ipyleaflet.Choropleth(
        style={'fillOpacity': 0.6, "weight": 0},
        geo_data=geo_data,
        choro_data=choro_data,
    )
    def on_hover(**kwargs):
        properties = kwargs.get("feature", {}).get("properties")
        id = kwargs.get("feature", {}).get("id")
        if not properties:
            return
        label.value=f"""
        <b>Tract GEOID: </b>{id} <br>
        <b>Number of {suffix} entitlements: </b> {properties[suffix]:,g} <br>
        <b>Median Household Income: </b> {properties["medhhincome"]:,g} <br>
        <b>Population Density: </b> {properties["density"]:,g} <br>
        <b>Percent Population White non-Hispanic: </b> {properties["pct_whitenonhisp"]:.2f} <br>
        <b>Percent Population Renter: </b> {properties["pct_pop_renter"]:.2f} <br>
        """
    choro_layer.on_hover(on_hover)
    for l in m.layers:
        if isinstance(l, ipyleaflet.Choropleth):
            m.substitute_layer(l, choro_layer)
            break
    else:
        m.add_layer(choro_layer)

In [6]:
demo = ipywidgets.Output()

def demo_plot(df, suffix, min_year, max_year):
    labels = {
        "medhhincome": "Median Household Income ($)",
        "density": "Population Density (people/sqmi)",
        "pct_whitenonhisp": "Percent population White non-Hispanic",
        "pct_pop_renter": "Percent Population Renter",
    }
    to_plot = df[(df[suffix] != 0) & (df.year >= min_year) & (df.year <= max_year)]
    to_plot = to_plot.groupby(to_plot.index).agg({
        suffix: "sum",
        **{c: "first" for c in cols}
    }).fillna(0.0).reset_index()

    fig, axes = plt.subplots(1, len(cols), figsize=(16,4))
    axes[0].set_ylabel(f"Number of {suffix} per tract")
    for var, ax in zip(cols, axes):
        ax.scatter(to_plot[var], to_plot[suffix], alpha=0.1)
        ax.set_xlabel(labels[var])
    plt.close(fig)
    with demo:
        demo.clear_output(wait=True)
        display(fig)

In [7]:
years = list(range(2010, 2020))
year_slider = ipywidgets.SelectionRangeSlider(
    description="Years",
    options=years,
    value=(years[0], years[-1]),
    continuous_update=False,
)
suffix_dropdown = ipywidgets.Dropdown(description="Suffix")

display(year_slider)
display(suffix_dropdown)
display(m)
display(demo)

change_guard = False

def on_suffix_selection(*args):
    global change_guard
    if change_guard:
        return
    suffix = suffix_dropdown.value
    plot_entitlement(joined, tracts, suffix, year_slider.value[0], year_slider.value[1])
    demo_plot(joined, suffix, year_slider.value[0], year_slider.value[1])

    
def on_year_selection(*args):
    global change_guard
    year_condition = (
        (joined.year >= year_slider.value[0]) &
        (joined.year <= year_slider.value[1])
    )
    counts = joined.loc[year_condition, suffix_list].sum()
    # Sort by alphabetical or in descending value of counts?
    counts = counts.sort_index()
    old_val = suffix_dropdown.value 
    change_guard=True
    suffix_dropdown.options = [
        (f"{name} ({count:,} applications)", name) 
        for name,count in zip(counts.index, counts)
    ]
    if old_val in counts.index:
        suffix_dropdown.value = old_val
    else:
        suffix_dropdown.index = 0
    change_guard=False
    on_suffix_selection()

on_year_selection()
suffix_dropdown.observe(on_suffix_selection, names="value")
year_slider.observe(on_year_selection, names="value")

SelectionRangeSlider(continuous_update=False, description='Years', index=(0, 9), options=(2010, 2011, 2012, 20…

Dropdown(description='Suffix', options=(), value=None)

Map(center=[34.07996230865876, -118.31123326410754], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Output()

In [8]:
case_dropdown = ipywidgets.Dropdown(
    description="Outlier cases",
    options=tuple(
        (f"{r[1]} ({r[2]} parcels)", r[0]) for r in
        big_cases.groupby("CASE_ID").agg(
            {"CASE_NUMBER": "first", "AIN": "count"}
        ).sort_values("AIN", ascending=False).itertuples()
    )
)

outlier_output = ipywidgets.Output()

display(case_dropdown)
display(outlier_output)


def plot_case(case_id):
    to_map = geopandas.GeoDataFrame(
        big_cases[big_cases.CASE_ID == case_id].groupby("GEOID").agg({
            "AIN": "count",
            "PROJECT_DESCRIPTION": "first"
        }).merge(
            tracts,
            how="right",
            left_index=True,
            right_on="GEOID"
        )
    )
    description = to_map.PROJECT_DESCRIPTION.dropna().iloc[0]
    with outlier_output:
        outlier_output.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12,12))
        to_map.fillna(0).plot(column="AIN",legend=True, ax=ax, cmap="plasma")
        plt.close()
        print(description)
        display(fig)

def on_case_selection(*args):
    plot_case(case_dropdown.value)

case_dropdown.observe(on_case_selection, names=["value"])
on_case_selection()

Dropdown(description='Outlier cases', options=(('CPC-2010-2278-GPA (19851 parcels)', 180067), ('CPC-2018-3723-…

Output()

In [9]:
big_cases.columns.values

array(['CASE_ID', 'CASE_NUMBER', 'FILE_DATE', 'APPLICATION_ID',
       'CASE_SEQUENCE_NUMBER', 'CASE_YEAR_NUMBER', 'PARENT_CASE_ID',
       'CASE_ACTION_ID', 'ADDRESS', 'COUNCIL_DISTRICT', 'PIN',
       'PLAN_AREA', 'BOE_DISTRICT', 'APC_AREA', 'CENSUS_TRACT', 'ZONING',
       'AIN', 'EXPEDITED_CASE', 'INCIDENTAL_CASE', 'PROJECT_DESCRIPTION',
       'APPEAL_HEARING_DATE', 'APPEAL_DECISION_DATE', '1A', '2A', 'AA',
       'AC', 'ACI', 'ADD1', 'ADM', 'ADU', 'AIC', 'APCC', 'APCE', 'APCH',
       'APCNV', 'APCS', 'APCSV', 'APCW', 'BL', 'BSA', 'CASP', 'CC', 'CC1',
       'CC3', 'CCMP', 'CDO', 'CDP', 'CE', 'CEX', 'CHC', 'CLQ', 'CM', 'CN',
       'COA', 'COC', 'CPC', 'CPIOA', 'CPIOC', 'CPIOE', 'CR', 'CRA', 'CU',
       'CUB', 'CUC', 'CUE', 'CUW', 'CUX', 'CUZ', 'CWC', 'CWNC', 'DA',
       'DB', 'DD', 'DEM', 'DI', 'DIR', 'DPS', 'DRB', 'EAF', 'ELD', 'EXT',
       'EXT2', 'EXT3', 'EXT4', 'F', 'GB', 'GPA', 'GPAJ', 'HCA', 'HCM',
       'HPO', 'INT', 'M1', 'M10', 'M11', 'M2', 'M3', 'M6', 'M7', 'M8',
 

In [10]:
big_cases.groupby("CASE_ID").agg({
    "CASE_NBR": "first",
    "PROJ_DESC_TXT": "first",
    "AIN": "count"
}).sort_values("AIN", ascending=False).rename(
    columns={"PROJ_DESC_TXT": "Description", "AIN": "Parcels"}
).style

SpecificationError: Column(s) ['CASE_NBR', 'PROJ_DESC_TXT'] do not exist